In [131]:
# From https://stackoverflow.com/a/5849861
import time
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        print('Elapsed [{}]: {}ms'.format(self.name, round((time.time() - self.tstart) * 1e3)))

In [132]:
def transpose(J2I) :
    I2J = {
        i : set()
        for I in J2I.values() for i in I
    }

    for (j, I) in J2I.items() :
        for i in I :
            I2J[i].add(j)
    
    return I2J
            
def rank_ref(J2I) :
    I2J = transpose(J2I)
            
    rank = 0
    while J2I and I2J :
        # Default pivot option
        I = next(iter(J2I.values()))
        J = I2J[next(iter(I))]

        # An alternative option
        JA = next(iter(I2J.values()))
        IA = J2I[next(iter(JA))]

        # Choose the option with fewer indices
        # (reducing the number of loops below)
        if ((len(IA) + len(JA)) < (len(I) + len(J))) : (I, J) = (IA, JA)

        for i in I : 
            I2J[i] = J.symmetric_difference(I2J[i])
            if not I2J[i] : del I2J[i]

        for j in J : 
            J2I[j] = I.symmetric_difference(J2I[j])
            if not J2I[j] : del J2I[j]

        rank += 1
    
    return rank

In [133]:
def rank1(J2I) :
    I2J = transpose(J2I)

    pivots = list(J2I.keys())
    
    while pivots :
        q = pivots.pop(0)
        if (not (q in J2I)) : continue
        
        I = J2I[q]
        p = next(iter(I))
        J = I2J[p]
        J.remove(q)

        for i in I : 
            I2J[i] = J.symmetric_difference(I2J[i])
            #if (q in I2J[i]) : I2J[i].remove(q)
            if not I2J[i] : del I2J[i]

        for j in J : 
            J2I[j] = I.symmetric_difference(J2I[j])
            if not J2I[j] : del J2I[j]
    
    return J2I

In [134]:
def nnz(J2I) :
    return sum(len(I) for I in J2I.values())

def rank2(J2I) :
    from math import ceil
    
    # Partition J2I
    
    # https://stackoverflow.com/a/12988416/3609568
    A = dict(list(J2I.items())[ceil(len(J2I)/2):])
    B = dict(list(J2I.items())[:ceil(len(J2I)/2)])
    
    with Timer("rank1(A)") :
        A = rank1(A)
    with Timer("rank1(B)") :
        B = rank1(B)
    
    AB = transpose({ **A, **B })
    with Timer("rank1(A+B)") :
        rank = len(rank1(AB))
    
    print("Len before/after/rank: {}/{}/{}".format(len(J2I), len(A) + len(B), rank))
    print("NNZ before/after:      {}/{}".format(nnz(J2I), nnz(A) + nnz(B)))
    
    with Timer("rank_ref") :
        rank0 = rank_ref(J2I)
    
    assert(rank == rank0)
    
    print("")
    
    return rank

In [135]:
def betti_bin(C) :
    from itertools import combinations as subcliques
    
    # Each maximal clique as a sorted tuple
    C = [tuple(sorted(mc)) for mc in C]
    
    all_J2I = []
    
    # (k-1)-chain group
    Sl = dict()
    # Maximal clique size
    K = max(len(s) for s in C)
    # Iterate over the dimension
    for k in range(0, K) :
        # Get all (k+1)-cliques, i.e. k-simplices, from max cliques mc
        Sk = set(c for mc in C for c in subcliques(mc, k+1))
        # Check that each simplex is in increasing order
        assert(all((list(s) == sorted(s)) for s in Sk))
        # Assign an ID to each simplex, in lexicographic order
        # (This ordering makes subsequent computations faster)
        Sk = dict(zip(sorted(Sk), range(0, len(Sk))))
        
        # Sk is now a representation of the k-chain group
        
        # J2I is a mapped representation of the boundary operator
        # (Understood to have boolean entries instead of +1 / -1)
        
        J2I = {
            j : set(Sl[s] for s in subcliques(ks, k) if s)
            for (ks, j) in Sk.items()
        }
        
        Sl = Sk
        
        all_J2I.append(J2I)
    
    assert(len(all_J2I) == K)
    
    
    # Betti numbers
    B = []
    for (k, J2I) in enumerate(all_J2I) :
        # "Default" Betti number (if rank is 0)
        B.append(len(J2I))
        
        if (k == 0) : continue
        
        #rank = rank1(J2I)[0]
        rank = rank2(J2I)

        B[k-1] -= rank
        B[k]   -= rank
    
    # Remove trailing zeros
    while B and (B[-1] == 0) : B.pop()
    
    return B

In [136]:
import networkx as nx
import numpy as np

for i in range(0, 3) :
    G = nx.gnp_random_graph(50, 0.5, seed=i)
    B = betti_bin(nx.find_cliques(G))
    print(B)

Elapsed [rank1(A)]: 1ms
Elapsed [rank1(B)]: 1ms
Elapsed [rank1(A+B)]: 0ms
Len before/after/rank: 615/84/49
NNZ before/after:      1230/168
Elapsed [rank_ref]: 1ms

Elapsed [rank1(A)]: 25ms
Elapsed [rank1(B)]: 45ms
Elapsed [rank1(A+B)]: 58ms
Len before/after/rank: 2490/896/566
NNZ before/after:      7470/5199
Elapsed [rank_ref]: 70ms

Elapsed [rank1(A)]: 200ms
Elapsed [rank1(B)]: 229ms
Elapsed [rank1(A+B)]: 4616ms
Len before/after/rank: 3771/2598/1920
NNZ before/after:      15084/52750
Elapsed [rank_ref]: 732ms

Elapsed [rank1(A)]: 24ms
Elapsed [rank1(B)]: 17ms
Elapsed [rank1(A+B)]: 171ms
Len before/after/rank: 2276/1974/1771
NNZ before/after:      11380/26526
Elapsed [rank_ref]: 59ms

Elapsed [rank1(A)]: 2ms
Elapsed [rank1(B)]: 2ms
Elapsed [rank1(A+B)]: 6ms
Len before/after/rank: 550/521/505
NNZ before/after:      3300/4464
Elapsed [rank_ref]: 4ms

Elapsed [rank1(A)]: 0ms
Elapsed [rank1(B)]: 0ms
Elapsed [rank1(A+B)]: 0ms
Len before/after/rank: 45/45/45
NNZ before/after:      315/330
El